In [161]:
# %matplotlib
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

import pandas as pd
import numpy as np 



In [162]:
PATH = "../results/headon-v-collisions.csv"
with open(PATH, "r") as f:
    print(f.readline())

step,contactLevel,contactEvents,drone_speed,drone_y_pos,heli_speed,drone_response_distance,drone_ascent_rate



In [163]:
all_exps = pd.read_csv(PATH)#.drop_duplicates()
print(all_exps.shape[0])
# all_exps['drone_speed'] = all_exps['drone_speed']/1.46667
# all_exps['heli_speed'] = all_exps['heli_speed']/1.46667
# all_exps['drone_ascent_rate'] = all_exps['drone_ascent_rate'] * 180 / np.pi
all_exps['is_violation'] = np.where(
    (all_exps['contactLevel'] != 'none') | ("violation" in all_exps['contactEvents']) | ("collision" in all_exps['contactEvents']), 
    1, 0)
# violations = all_exps[(all_exps['contactLevel'] == 'violation') | (all_exps['contactLevel'] == 'collision')]
# violations.columns
all_exps = all_exps.round(2)
all_exps
all_exps[all_exps.contactLevel == 'none'].count()

42075


step                       15614
contactLevel               15614
contactEvents              15614
drone_speed                15614
drone_y_pos                15614
heli_speed                 15614
drone_response_distance    15614
drone_ascent_rate          15614
is_violation               15614
dtype: int64

In [164]:
def add_temp_legend(values, element, ax):
    np.unique(values)[::-1].sort()
    colors = [ element.cmap(element.norm(value)) for value in values]
    # create a patch (proxy artist) for every color 
    patches = [ mpatches.Patch(color=colors[i], label="{l} violations".format(l=values[i]) ) for i in range(len(values)) ]
        
    ax.legend(handles=patches, bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.0)

def heatmap(df: pd.DataFrame, x_key: str, y_key: str, title:str, **kwargs):
    df = df.groupby([x_key,y_key], as_index=False).agg({'is_violation': np.sum})
    df = df.pivot(columns=x_key, index=y_key, values="is_violation")
    
    fig, ax = plt.subplots()
    im = ax.imshow(df.to_numpy(), **kwargs)
    ax.set_title(title)
    ax.set_xlabel(x_key)
    ax.set_ylabel(y_key)
    
    ax.set_xticks(np.arange(0, len(df.columns), 1))
    ax.set_yticks(np.arange(0, len(df.index), 1))
    ax.set_xticklabels(list(map(lambda v: round(v, 2), df.columns)))
    ax.set_yticklabels(list(map(lambda v: round(v, 2), df.index)))
    
    values = np.unique(df.to_numpy().flatten())
    
    add_temp_legend(values, im, ax)
    
    
    
def isolate(df: pd.DataFrame, cols: dict): # keys:list[str], values:list
    exp = None
    for k, v in cols.items():
        if exp is None:
            exp = df[k] == v
        else:
            exp &= df[k] == v
            
    return df[exp]
    

In [165]:
heatmap(all_exps, "drone_speed", "drone_ascent_rate", "head-on collisions")

Commenting on the figure above, top-right is the highest turn radius, which means
that the drone will be closer to the helicopter

In [166]:
all_exps.drone_response_distance.unique()

array([5000., 5300., 5600., 5900., 6200., 6500., 6800., 7100., 7400.,
       7700., 8000.])

In [167]:
# tmp = all_exps[all_exps['drone_y_pos'] <= 2000]#[all_exps['drone_horizontal_turn_rate'] > 35]
# tmp = isolate(tmp, {"drone_horizontal_turn_rate": 30})
heatmap(all_exps, "drone_response_distance", "drone_ascent_rate", "head-on collisions")

Commenting on the figure above, the bottom left

3D, visualizing speed, rate of turn, and vis distance all in one

In [168]:
def plot_point_cloud(df: pd.DataFrame, x_key: str, y_key: str, z_key: str, title: str, point_radius: float = None):
    fig = plt.figure()
    fig.set_figheight(10)
    fig.set_figwidth(10)
    ax = fig.add_subplot(projection='3d')
    
    x = df[x_key].unique()
    x.sort()
    y = df[y_key].unique()
    y.sort()
    z = df[z_key].unique()
    z.sort()
    
    points = []
    temps = []
    
    
    for zi in z:
        for yi in y:
            for xi in x:
                points.append((xi, yi, zi))
                temps.append(isolate(df, {x_key:xi, y_key:yi, z_key:zi}).size)

    s = point_radius ** 2 if point_radius else None

    print(len(points), points)

    scat = ax.scatter(*np.array(points).T, c=temps, alpha=0.7, s=s)
    ax.set_xlabel(x_key)
    ax.set_ylabel(y_key)
    ax.set_zlabel(z_key)
    ax.set_title(title)
    
        
    
    add_temp_legend(np.unique(np.array(temps[::-1]).flatten()), scat, ax)
    

In [169]:
all_exps.columns

Index(['step', 'contactLevel', 'contactEvents', 'drone_speed', 'drone_y_pos',
       'heli_speed', 'drone_response_distance', 'drone_ascent_rate',
       'is_violation'],
      dtype='object')

In [170]:
%matplotlib

Using matplotlib backend: TkAgg


In [171]:
# tmp = all_exps[all_exps['drone_y_pos'] <= 2000]
tmp = isolate(all_exps, {
    "is_violation": 1, 
    # "heli_speed": 118.7998,
    # "drone_y_pos":4000.0,
})
plot_point_cloud(tmp, "drone_speed", "drone_ascent_rate",
"drone_response_distance", "Drone Visual Distance Scatter", point_radius=20)
# tmp.count()
print("Axes:", all_exps.drone_ascent_rate.unique(), all_exps.drone_speed.unique(), sep="\n")

all_exps[
    (all_exps['drone_speed'] >= 36) &
    (all_exps['drone_speed'] <= 44) &
    (all_exps['drone_ascent_rate'] <= -14.2) &
    (all_exps['drone_ascent_rate'] <= -8.2) 
].describe()

#speed: [36, 44]
#asc: [-14.2,  -8.2]

1683 [(36.0, -14.2, 5000.0), (37.0, -14.2, 5000.0), (38.0, -14.2, 5000.0), (39.0, -14.2, 5000.0), (40.0, -14.2, 5000.0), (41.0, -14.2, 5000.0), (42.0, -14.2, 5000.0), (43.0, -14.2, 5000.0), (44.0, -14.2, 5000.0), (36.0, -13.82, 5000.0), (37.0, -13.82, 5000.0), (38.0, -13.82, 5000.0), (39.0, -13.82, 5000.0), (40.0, -13.82, 5000.0), (41.0, -13.82, 5000.0), (42.0, -13.82, 5000.0), (43.0, -13.82, 5000.0), (44.0, -13.82, 5000.0), (36.0, -13.45, 5000.0), (37.0, -13.45, 5000.0), (38.0, -13.45, 5000.0), (39.0, -13.45, 5000.0), (40.0, -13.45, 5000.0), (41.0, -13.45, 5000.0), (42.0, -13.45, 5000.0), (43.0, -13.45, 5000.0), (44.0, -13.45, 5000.0), (36.0, -13.08, 5000.0), (37.0, -13.08, 5000.0), (38.0, -13.08, 5000.0), (39.0, -13.08, 5000.0), (40.0, -13.08, 5000.0), (41.0, -13.08, 5000.0), (42.0, -13.08, 5000.0), (43.0, -13.08, 5000.0), (44.0, -13.08, 5000.0), (36.0, -12.7, 5000.0), (37.0, -12.7, 5000.0), (38.0, -12.7, 5000.0), (39.0, -12.7, 5000.0), (40.0, -12.7, 5000.0), (41.0, -12.7, 5000.0), (

,step,drone_speed,drone_y_pos,heli_speed,drone_response_distance,drone_ascent_rate,is_violation
count,2475.0,2475.000000,2475.000000,2475.000000,2475.000000,2475.0,2475.000000
mean,520.0,40.000000,3068.182000,168.666000,6500.000000,-14.2,0.266263
std,0.0,2.582511,241.108486,10.372524,948.875009,0.0,0.442093
min,520.0,36.000000,2727.270000,154.000000,5000.000000,-14.2,0.000000
25%,520.0,38.000000,2897.730000,161.330000,5600.000000,-14.2,0.000000
50%,520.0,40.000000,3068.180000,168.670000,6500.000000,-14.2,0.000000
75%,520.0,42.000000,3238.640000,176.000000,7400.000000,-14.2,1.000000
max,520.0,44.000000,3409.090000,183.330000,8000.000000,-14.2,1.000000


In [172]:
# isolate(all_exps, {"is_violation": 1}).describe()
all_exps.describe()

,step,drone_speed,drone_y_pos,heli_speed,drone_response_distance,drone_ascent_rate,is_violation
count,42075.0,42075.00000,42075.000000,42075.000000,42075.000000,42075.000000,42075.000000
mean,520.0,40.00000,3068.182000,168.666000,6500.000000,-11.198824,0.628901
std,0.0,2.58202,241.062637,10.370552,948.694572,1.837862,0.483105
min,520.0,36.00000,2727.270000,154.000000,5000.000000,-14.200000,0.000000
25%,520.0,38.00000,2897.730000,161.330000,5600.000000,-12.700000,0.000000
50%,520.0,40.00000,3068.180000,168.670000,6500.000000,-11.200000,1.000000
75%,520.0,42.00000,3238.640000,176.000000,7400.000000,-9.700000,1.000000
max,520.0,44.00000,3409.090000,183.330000,8000.000000,-8.200000,1.000000


In [173]:
all_exps.heli_speed.unique()

array([154.  , 161.33, 168.67, 176.  , 183.33])

In [174]:
# best = isolate(all_exps, {'is_violation': 0, 'drone_y_pos': 4000, 'drone_response_distance': 4200, 'heli_speed': 109})
best = isolate(all_exps, 
    {
        'is_violation': 1, 
        # 'drone_y_pos': 4000, 
        # "drone_horizontal_turn_rate": 12.0, 
        "drone_speed": 40.6153, 
        # "drone_response_distance": 6250,
        "heli_speed": 114.7383
})
# best = isolate(all_exps, {'is_violation': 0, 'drone_y_pos': 4000, 'drone_speed': 36, 'heli_speed': 118})
# best.drone_horizontal_turn_rate.min()
isolate(all_exps, {
    'is_violation': 0,
    # 'drone_y_pos': 4000, 
    "drone_speed": 38.0, 
    "heli_speed": 115
}).drone_response_distance.min()
# best.heli_speed
# speeds = best.drone_speed.to_numpy()
# speeds.sort()
# speeds 
# best[(best.drone_response_distance == 4200)]



nan

In [175]:
90/13

6.923076923076923